In [1]:
import math
import itertools
import numpy

def euclidean_distance(x1, y1, x2, y2):
    """ユークリッド距離を計算
    """
    t1 = math.pow(x1 - x2, 2)
    t2 = math.pow(y1 - y2, 2)
    t3 = math.pow(t1 + t2, 0.5)
    return t3

class Rectangle(object):
    def __init__(self, x_min, y_min, x_max, y_max, label):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.label = label
        self.shape = None

    def get_distance(self, other):
        """矩形間の距離を計算
            矩形に重なりがある場合は0を返す。そうでなければ矩形間の最短距離を返す。
        """
        if self._rectangle_collision(other): # 重なりがある場合
            return 0
        if self._line_collision(other, "vertical"): # 縦の線分で重なる
            d1 = math.fabs(self.x_min - other.x_max)
            d2 = math.fabs(self.x_max - other.x_min)
            return min(d1, d2)
        elif self._line_collision(other, "horizontal"): # 横の線分で重なる
            d1 = math.fabs(self.y_min - other.y_max)
            d2 = math.fabs(self.y_max - other.y_min)
            return min(d1, d2)
        else: # 斜めの位置関係にある
            points1 = [
                (self.x_min, self.y_min),
                (self.x_min, self.y_max),
                (self.x_max, self.y_min),
                (self.x_max, self.y_max)
            ]
            points2 = [
                (other.x_min, other.y_min),
                (other.x_min, other.y_max),
                (other.x_max, other.y_min),
                (other.x_max, other.y_max)
            ]
            d_list = [
                euclidean_distance(
                    p1[0], p1[1], p2[0], p2[1]
                ) for p1, p2 in itertools.product(points1, points2)
            ]
            return numpy.array(d_list).min()
        
        
    def _rectangle_collision(self, other):
        """矩形として重なりがあるかどうか判定
        """
        cx1 = (self.x_min + self.x_max) / 2
        cy1 = (self.y_min + self.y_max) / 2
        width1 = math.fabs(cx1 - self.x_min) * 2
        height1 = math.fabs(cy1 - self.y_min) * 2
        
        cx2 = (other.x_min + other.x_max) / 2
        cy2 = (other.y_min + other.y_max) / 2
        width2 = math.fabs(cx2 - other.x_min) * 2
        height2 = math.fabs(cy2 - other.y_min) * 2
        flag_w = math.fabs(cx1 - cx2) < width1 / 2 + width2 / 2
        flag_h = math.fabs(cy1 - cy2) < height1 / 2 + height2 / 2
        return flag_w and flag_h
    
        
    def _line_collision(self, other, shape):
        """線分として重なりがあるかどうか判定
        """
        assert shape in ("vertical", "horizontal")
        if shape == "vertical": # 縦
            v11 = self.y_min
            v12 = self.y_max
            v21 = other.y_min
            v22 = other.y_max
        elif shape == "horizontal": # 横
            v11 = self.x_min
            v12 = self.x_max
            v21 = other.x_min
            v22 = other.x_max
        flag_1 = (v11 <= v21 <= v12) or (v21 <= v11 and v12 <= v22)
        flag_2 = (v21 <= v11 <= v22) or (v11 <= v21 and v22 <= v12)
        return flag_1 or flag_2

In [2]:
def test_rectangle_collision():
    rec1 = Rectangle(0, 0, 10, 10, "")
    rec2 = Rectangle(0, 0, 10, 10, "")
    assert rec1._rectangle_collision(rec2)
    assert rec2._rectangle_collision(rec1)
    
    rec3 = Rectangle(10, 10, 20, 20, "")
    assert not rec1._rectangle_collision(rec3)
    assert not rec3._rectangle_collision(rec1)
    
    rec4 = Rectangle(9, 9, 20, 20, "")
    assert rec1._rectangle_collision(rec4)
    assert rec4._rectangle_collision(rec1)
    
    rec5 = Rectangle(11, 11, 20, 20, "")
    assert not rec1._rectangle_collision(rec5)
    assert not rec5._rectangle_collision(rec1)

    rec6 = Rectangle(5, 0, 10, 10, "")
    assert rec1._rectangle_collision(rec6)
    assert rec6._rectangle_collision(rec1)

    rec7 = Rectangle(5, 5, 6, 6, "")
    assert rec1._rectangle_collision(rec7)
    assert rec7._rectangle_collision(rec1)

def test_line_collision():
    rec1 = Rectangle(0, 0, 10, 10, "")
    rec2 = Rectangle(0, 0, 10, 10, "")
    
    assert rec1._line_collision(rec2, "vertical")
    assert rec2._line_collision(rec1, "vertical")
    assert rec1._line_collision(rec2, "horizontal")
    assert rec2._line_collision(rec1, "horizontal")

    rec3 = Rectangle(11, 0, 20, 10, "")   
    assert rec1._line_collision(rec3, "vertical")
    assert rec3._line_collision(rec1, "vertical")
    assert not rec1._line_collision(rec3, "horizontal")
    assert not rec3._line_collision(rec1, "horizontal")

    rec4 = Rectangle(5, 30, 20, 40, "")
    assert not rec1._line_collision(rec4, "vertical")
    assert not rec4._line_collision(rec1, "vertical")
    assert rec1._line_collision(rec4, "horizontal")
    assert rec4._line_collision(rec1, "horizontal")

    rec5 = Rectangle(15, 30, 20, 40, "")
    assert not rec1._line_collision(rec5, "vertical")
    assert not rec5._line_collision(rec1, "vertical")
    assert not rec1._line_collision(rec5, "horizontal")
    assert not rec5._line_collision(rec1, "horizontal")

    rec6 = Rectangle(10, -2, 11, -1, "")
    assert not rec1._line_collision(rec6, "vertical")
    assert not rec6._line_collision(rec1, "vertical")
    assert rec1._line_collision(rec6, "horizontal")
    assert rec6._line_collision(rec1, "horizontal")

def test_get_distance():
    # 重なるパターン
    rec1 = Rectangle(0, 0, 10, 10, "")
    assert rec1.get_distance(rec1) == 0
    
    rec2 = Rectangle(0, 0, 10, 20, "")
    rec3 = Rectangle(20, 5, 22, 10, "")
    assert rec2.get_distance(rec3) == 10
    assert rec3.get_distance(rec2) == 10
    
    rec4 = Rectangle(14, -5, 20, 7, "")
    assert rec2.get_distance(rec4) == 4
    assert rec4.get_distance(rec2) == 4
    
    rec5 = Rectangle(13, 24, 25, 50, "")
    assert rec2.get_distance(rec5) == 5
    assert rec5.get_distance(rec2) == 5

    rec6 = Rectangle(0, 22, 10, 32, "")
    assert rec2.get_distance(rec6) == 2
    assert rec6.get_distance(rec2) == 2

    rec7 = Rectangle(0, 0, 20, 10, "")
    rec8 = Rectangle(-5, 12, 5, 14, "")
    assert rec7.get_distance(rec8) == 2
    assert rec8.get_distance(rec7) == 2

    rec9 = Rectangle(-5, 15, 25, 20, "")
    assert rec7.get_distance(rec9) == 5
    assert rec9.get_distance(rec7) == 5
    
    rec10 = Rectangle(-30, 13, -4, 20, "")
    assert rec7.get_distance(rec10) == 5
    assert rec10.get_distance(rec7) == 5
    
    rec11 = Rectangle(25, 0, 50, 10, "")
    assert rec7.get_distance(rec11) == 5
    assert rec11.get_distance(rec7) == 5
    
    rec12 = Rectangle(0, 10, 10, 20, "")
    rec13 = Rectangle(13, -5, 40, 6, "")
    assert rec13.get_distance(rec12) == 5
    assert rec12.get_distance(rec13) == 5


In [3]:
test_rectangle_collision()
test_line_collision()
test_get_distance()